# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Hyundai_crew_predictor').getOrCreate()

In [66]:
cruise_ship_data=spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [67]:
cruise_ship_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [68]:
cruise_ship_data.count()

158

In [69]:
#checking for missing values
cruise_ship_data=cruise_ship_data.dropna()

In [70]:
cruise_ship_data.count()

158

In [71]:
#checking data correlation
cruise_ship_data.corr('length','crew')

0.895856627101658

In [95]:
cruise_ship_data.corr('Age','crew')

-0.5306565039638852

In [94]:
cruise_ship_data.corr('Tonnage','crew')

0.9275688115449388

In [112]:
cruise_ship_data.select('Tonnage','passengers','crew').show()

+------------------+----------+----+
|           Tonnage|passengers|crew|
+------------------+----------+----+
|30.276999999999997|      6.94|3.55|
|30.276999999999997|      6.94|3.55|
|            47.262|     14.86| 6.7|
|             110.0|     29.74|19.1|
|           101.353|     26.42|10.0|
|            70.367|     20.52| 9.2|
|            70.367|     20.52| 9.2|
|            70.367|     20.56| 9.2|
|            70.367|     20.52| 9.2|
|110.23899999999999|      37.0|11.5|
|             110.0|     29.74|11.6|
|            46.052|     14.52| 6.6|
|            70.367|     20.52| 9.2|
|            70.367|     20.52| 9.2|
|              86.0|     21.24| 9.3|
|             110.0|     29.74|11.6|
|              88.5|     21.24|10.3|
|            70.367|     20.52| 9.2|
|              88.5|     21.24| 9.3|
|            70.367|     20.52| 9.2|
+------------------+----------+----+
only showing top 20 rows



In [72]:
#Encoding for including cruise_line which is mentioned as a significant feature
from pyspark.sql.functions import col, split
cruise_ship_data = cruise_ship_data.withColumn("Cruise_line_Array", split(col("Cruise_line")," "))

In [73]:
from pyspark.ml.feature import CountVectorizer
cnt_vectorizer = CountVectorizer(inputCol='Cruise_line_Array',outputCol='Encoded_Cruiseline') 
vect_model=cnt_vectorizer.fit(cruise_ship_data)
cruise_ship_data=vect_model.transform(cruise_ship_data)


In [74]:
cruise_ship_data.select('Encoded_Cruiseline').show()

+------------------+
|Encoded_Cruiseline|
+------------------+
|   (20,[17],[1.0])|
|   (20,[17],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
|    (20,[1],[1.0])|
+------------------+
only showing top 20 rows



In [75]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [76]:
assembler=VectorAssembler(inputCols=['Encoded_Cruiseline',
 'Tonnage',
 'passengers',
 'length',
 'cabins'],outputCol="features"
                         )

In [77]:
feature_out=assembler.transform(cruise_ship_data)

In [78]:
final_data = feature_out.select("features",'crew')

In [79]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [80]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              122|
|   mean|7.892540983606561|
| stddev|3.530058544097073|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [81]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                36|
|   mean|7.4608333333333325|
| stddev|3.4396182304768312|
|    min|              0.88|
|    max|              13.6|
+-------+------------------+



In [82]:
from pyspark.ml.regression import LinearRegression

In [83]:
regression_model=LinearRegression(labelCol='crew')

In [84]:
linmodel=regression_model.fit(train_data,)

In [85]:
print("Coefficients: {} Intercept: {}".format(linmodel.coefficients,linmodel.intercept))

Coefficients: [-1.1470109008662708,0.45433707902029496,-0.047237202625684684,-0.6486263433730788,0.4444463850894083,-0.32072708477715445,0.6718056705287264,-0.1316045282323109,-0.09683840273159539,1.4666534753865976,0.7605599685921137,0.24784707574713377,-0.2664272969794378,0.6024541610013375,0.5295325568108677,-0.05971590884569131,0.553959712451918,0.08131522709565424,0.06684798617224645,-0.03398774131588001,0.020302623997969567,-0.08071160998468214,0.5515813041855745,0.6093084610782367] Intercept: -2.0253171832391192


In [86]:
test_results = linmodel.evaluate(test_data)

In [87]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.09256844654886365|
| 0.08038227732729908|
|   0.337911066258501|
|-0.00952555319504...|
|  0.0675811444547012|
| -1.6033798005630242|
| 0.37463524364360756|
| -0.4740608634096164|
|-0.04627708180254153|
|  0.7875101408068321|
|  0.9245939976638056|
| 0.09840002192196273|
|-0.35059633511981936|
|  0.6834827256037697|
|  0.6271595699873442|
|  0.6610220750448468|
| 0.05492729565891796|
| 0.09878084487454508|
|  0.5320343897519018|
| -0.1989330910318312|
+--------------------+
only showing top 20 rows



In [88]:
unlabeled_data = test_data.select('features')

In [89]:
predictions = linmodel.transform(unlabeled_data)

In [90]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(24,[0,20,21,22,2...| 8.127431553451137|
|(24,[0,20,21,22,2...| 8.139617722672702|
|(24,[0,20,21,22,2...| 8.342088933741499|
|(24,[0,20,21,22,2...| 13.60952555319504|
|(24,[1,20,21,22,2...| 9.132418855545298|
|(24,[1,20,21,22,2...|10.903379800563025|
|(24,[2,20,21,22,2...|3.3553647563563924|
|(24,[2,20,21,22,2...| 5.674060863409617|
|(24,[2,20,21,22,2...| 9.046277081802542|
|(24,[2,20,21,22,2...|11.212489859193168|
|(24,[2,20,21,22,2...|11.455406002336195|
|(24,[2,20,21,22,2...|11.901599978078037|
|(24,[2,20,21,22,2...| 11.35059633511982|
|(24,[3,20,21,22,2...|  4.61651727439623|
|(24,[3,20,21,22,2...| 5.252840430012656|
|(24,[3,20,21,22,2...| 5.778977924955154|
|(24,[3,20,21,22,2...| 5.945072704341082|
|(24,[4,20,21,22,2...|10.991219155125455|
|(24,[5,20,21,22,2...|3.3179656102480983|
|(24,[5,20,21,22,2...|  9.39893309103183|
+--------------------+------------

In [108]:

predictions.select("features").rdd.flatMap(list).collect()[3]


SparseVector(24, {0: 1.0, 20: 158.0, 21: 43.7, 22: 11.12, 23: 18.0})

In [91]:
#testing model performance using RMSE
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
#the model performs well indicated by the MSE value

RMSE: 0.5892819964818984
MSE: 0.34725327137769213


In [114]:
cruise_ship_data.select(
 'Tonnage',
 'passengers',
 'length',
 'cabins','crew').show()

+------------------+----------+------+------+----+
|           Tonnage|passengers|length|cabins|crew|
+------------------+----------+------+------+----+
|30.276999999999997|      6.94|  5.94|  3.55|3.55|
|30.276999999999997|      6.94|  5.94|  3.55|3.55|
|            47.262|     14.86|  7.22|  7.43| 6.7|
|             110.0|     29.74|  9.53| 14.88|19.1|
|           101.353|     26.42|  8.92| 13.21|10.0|
|            70.367|     20.52|  8.55|  10.2| 9.2|
|            70.367|     20.52|  8.55|  10.2| 9.2|
|            70.367|     20.56|  8.55| 10.22| 9.2|
|            70.367|     20.52|  8.55|  10.2| 9.2|
|110.23899999999999|      37.0|  9.51| 14.87|11.5|
|             110.0|     29.74|  9.51| 14.87|11.6|
|            46.052|     14.52|  7.27|  7.26| 6.6|
|            70.367|     20.52|  8.55|  10.2| 9.2|
|            70.367|     20.52|  8.55|  10.2| 9.2|
|              86.0|     21.24|  9.63| 10.62| 9.3|
|             110.0|     29.74|  9.51| 14.87|11.6|
|              88.5|     21.24|

In [ ]:
#Why the model may perform well?
#There exists a linear relation between the features and the dependent variable.The
#correlation between the features and the dependent variable is high.Though correlation
#may not mean causation always. In this particular case we can get a sense of why a higher
# tonnage can reflect on higher number of crew memebers need in the ship.Similarly works for the other
#features